In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('/home/trishalaswain/ClientData/SCP/ML_DATA/BYRD_EXTENDED_SCP/BYRD_EXTENDED_FACS_AUG_OUTPUT.csv', sep='\t', index_col=False)

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
data.rename(columns = {'encounter':'encounter_id'}, inplace = True)

In [ ]:
codes = pd.read_excel('/home/trishalaswain/ClientData/SCP/ML_DATA/SCP_AUG_CODES.xlsx', index_col=False)

In [ ]:
codes.columns

In [ ]:
# codes.shape
# codes.rename(columns = {'id':'encounter_id'}, inplace = True)
# codes.rename(columns = {'accountNumber':'account_number'}, inplace = True)

In [ ]:
codes = codes[['facility_code', 'encounter_id', 'account_number', 'eandm_suggested', 'eandm_final']]

In [ ]:
inner_join = pd.merge(data, codes, on =['encounter_id', 'account_number', 'facility_code'], how ='inner')

In [ ]:
inner_join.encounter_id = inner_join.encounter_id.astype(int)
inner_join.account_number = inner_join.account_number.astype(int)
inner_join

In [ ]:
inner_join.ADMISSION_STATUS.value_counts()

In [ ]:
ml_df = inner_join.copy()

In [ ]:
ml_df['ADDENDUM'] = 0

In [ ]:
ml_df['ADMIT'] = 0

In [ ]:
ml_df.MDM_SENTS = ml_df.MDM_SENTS.apply(str.lower)

In [ ]:
for i, r in ml_df.iterrows():
    if 'addendum' in str(r['MDM_SENTS']):
        ml_df.loc[i, 'ADDENDUM'] = 1

In [ ]:
for i, r in ml_df.iterrows():
    if 'Admitted - HIGH' in str(r['ADMISSION_STATUS']):
        ml_df.loc[i, 'ADMIT'] = 1

In [ ]:
ml_df

In [ ]:
ml_df.ADMIT.sum()

In [ ]:
import re
for i,r in ml_df.iterrows():
    ml_df.loc[i,'clean_eandm_suggested'] = ','.join([str(elem) for elem in re.findall(r"\b\d{5}\b", str(r['eandm_suggested']))])
    ml_df.loc[i,'clean_eandm_final'] = ','.join([str(elem) for elem in re.findall(r"\b\d{5}\b", str(r['eandm_final']))])

In [ ]:
ml_df.info()

In [ ]:
ml_df.clean_eandm_suggested = ml_df.clean_eandm_suggested.astype('str')
ml_df.clean_eandm_final = ml_df.clean_eandm_final.astype('str')

In [ ]:
ml_df['TIME_CASES'] = 0
for i, r in ml_df.iterrows():
    if '99291' in r['clean_eandm_suggested'] or '99291' in r['clean_eandm_final']:
        ml_df.loc[i, 'TIME_CASES'] = 1

In [ ]:
ml_df['TIME_CASES'].sum()

In [ ]:
ml_df_final = ml_df[(ml_df.TIME_CASES == 0) & (ml_df.ADDENDUM == 0) & (ml_df.ADMIT == 0)].reset_index()

In [ ]:
ml_df_final

In [ ]:
ml_df_final.drop(['level_0', 'index'], inplace=True, axis=1)

In [ ]:
ml_df_final

In [ ]:
ml_df_final.to_excel('/home/trishalaswain/ClientData/SCP/ML_DATA/BYRD_EXTENDED_SCP/BYRD_EXTENDED_FACS_AUG_OUTPUT_FINAL.xlsx', index=False)